In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime

plt.rcParams['font.sans-serif'] = ['SimHei']  #设置中文字体为黑体
plt.rcParams['axes.unicode_minus'] = False #正常显示负号

pd.set_option('display.float_format',lambda x : '%.2f' % x)#pandas禁用科学计数法

#忽略警告
import warnings
warnings.filterwarnings('ignore')

### 1 加载数据

In [2]:
data = pd.read_csv('./data.csv',index_col=0,dtype={'category_id':'int64','user_id':'int64'},encoding='utf8')

In [3]:
data.head()

,event_time,order_id,product_id,category_id,category_code,brand,price,user_id,age,sex,local
0,2020-04-24 11:50:39 UTC,2294359932054536986,1515966223509089906,2268105426648171520,electronics.tablet,samsung,162.01,1515915625441993984,24.00,女,海南
1,2020-04-24 11:50:39 UTC,2294359932054536986,1515966223509089906,2268105426648171520,electronics.tablet,samsung,162.01,1515915625441993984,24.00,女,海南
2,2020-04-24 14:37:43 UTC,2294444024058086220,2273948319057183658,2268105430162997248,electronics.audio.headphone,huawei,77.52,1515915625447879424,38.00,女,北京
3,2020-04-24 14:37:43 UTC,2294444024058086220,2273948319057183658,2268105430162997248,electronics.audio.headphone,huawei,77.52,1515915625447879424,38.00,女,北京
4,2020-04-24 19:16:21 UTC,2294584263154074236,2273948316817424439,2268105471367840000,NaN,karcher,217.57,1515915625443148032,32.00,女,广东


In [7]:
data.shape

(564169, 11)

In [5]:
data.columns

Index(['event_time', 'order_id', 'product_id', 'category_id', 'category_code',
       'brand', 'price', 'user_id', 'age', 'sex', 'local'],
      dtype='object')

##### 字段解释 

1 event_time 下单时间
2 order_id  订单编号
3 product_id 产品编号
4 category_id 类别编号
5 category_code 类别
6 brand 品牌
7 price 价格
8 user_id 用户编号
9 age 年龄
10 sex 性别
11 local 省份

### 2 数据EDA

#### 2.1 日期数据处理 

因为原数据中的日期数据（下单时间）为精确到秒的格式，为了方便后续使用，将日期处理成日期列、月份、周几的格式

①处理日期列

In [5]:
#创建日期列
data['date'] = data.event_time.apply(lambda x : x.split(' ')[0])

In [6]:
#转化成日期格式
data['date'] = pd.to_datetime(data['date'])

②处理月份列

In [7]:
data['month'] = data.date.dt.month

③处理周几列：周日为0，周一为1，以此类推

In [8]:
data['weekday'] = data.date.apply(lambda x:x.strftime("%w"))

④删除event_time列 

In [9]:
del data['event_time']

In [10]:
data.head()

,order_id,product_id,category_id,category_code,brand,price,user_id,age,sex,local,date,month,weekday
0,2294359932054536986,1515966223509089906,2268105426648171520,electronics.tablet,samsung,162.01,1515915625441993984,24.00,女,海南,2020-04-24,4,5
1,2294359932054536986,1515966223509089906,2268105426648171520,electronics.tablet,samsung,162.01,1515915625441993984,24.00,女,海南,2020-04-24,4,5
2,2294444024058086220,2273948319057183658,2268105430162997248,electronics.audio.headphone,huawei,77.52,1515915625447879424,38.00,女,北京,2020-04-24,4,5
3,2294444024058086220,2273948319057183658,2268105430162997248,electronics.audio.headphone,huawei,77.52,1515915625447879424,38.00,女,北京,2020-04-24,4,5
4,2294584263154074236,2273948316817424439,2268105471367840000,NaN,karcher,217.57,1515915625443148032,32.00,女,广东,2020-04-24,4,5


#### 2.2 处理缺失数据

In [11]:
data.shape

(564169, 13)

In [12]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 564169 entries, 0 to 2633520
Data columns (total 13 columns):
 #   Column         Non-Null Count   Dtype         
---  ------         --------------   -----         
 0   order_id       564169 non-null  int64         
 1   product_id     564169 non-null  int64         
 2   category_id    564169 non-null  int64         
 3   category_code  434799 non-null  object        
 4   brand          536945 non-null  object        
 5   price          564169 non-null  float64       
 6   user_id        564169 non-null  int64         
 7   age            564169 non-null  float64       
 8   sex            564169 non-null  object        
 9   local          564169 non-null  object        
 10  date           564169 non-null  datetime64[ns]
 11  month          564169 non-null  int64         
 12  weekday        564169 non-null  object        
dtypes: datetime64[ns](1), float64(2), int64(5), object(5)
memory usage: 60.3+ MB


发现category_code（产品品类）和brand（品牌）两列存在数据缺失，因为产品品类缺失的比较多，直接用‘M’来替代缺失值；而brand缺失比较少，直接删除缺失值；

In [13]:
#使用M代替缺失值
data['category_code'] = data['category_code'].fillna("M")

In [14]:
#删除brand缺失行
data = data[data.brand.notnull()]

In [15]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 536945 entries, 0 to 2633520
Data columns (total 13 columns):
 #   Column         Non-Null Count   Dtype         
---  ------         --------------   -----         
 0   order_id       536945 non-null  int64         
 1   product_id     536945 non-null  int64         
 2   category_id    536945 non-null  int64         
 3   category_code  536945 non-null  object        
 4   brand          536945 non-null  object        
 5   price          536945 non-null  float64       
 6   user_id        536945 non-null  int64         
 7   age            536945 non-null  float64       
 8   sex            536945 non-null  object        
 9   local          536945 non-null  object        
 10  date           536945 non-null  datetime64[ns]
 11  month          536945 non-null  int64         
 12  weekday        536945 non-null  object        
dtypes: datetime64[ns](1), float64(2), int64(5), object(5)
memory usage: 57.4+ MB


#### 2.3 处理重复数据 

In [16]:
data.duplicated().sum()

634

存在634条重复数据，但是销售数据中可能存在一个用户在同天下单两次或多次的情况，所以不删除重复值，而是增加购买数量和总价格的列

In [17]:
#添加新列：购买数量
data = data.value_counts().reset_index().rename(columns={0:'buy_cnt'})

In [18]:
#添加新列：购买总金额
data['amount'] = data['price'] * data['buy_cnt']

In [19]:
data.columns

Index(['order_id', 'product_id', 'category_id', 'category_code', 'brand',
       'price', 'user_id', 'age', 'sex', 'local', 'date', 'month', 'weekday',
       'buy_cnt', 'amount'],
      dtype='object')

#### 2.4 查看数据是否存在异常 

In [20]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 536311 entries, 0 to 536310
Data columns (total 15 columns):
 #   Column         Non-Null Count   Dtype         
---  ------         --------------   -----         
 0   order_id       536311 non-null  int64         
 1   product_id     536311 non-null  int64         
 2   category_id    536311 non-null  int64         
 3   category_code  536311 non-null  object        
 4   brand          536311 non-null  object        
 5   price          536311 non-null  float64       
 6   user_id        536311 non-null  int64         
 7   age            536311 non-null  float64       
 8   sex            536311 non-null  object        
 9   local          536311 non-null  object        
 10  date           536311 non-null  datetime64[ns]
 11  month          536311 non-null  int64         
 12  weekday        536311 non-null  object        
 13  buy_cnt        536311 non-null  int64         
 14  amount         536311 non-null  float64       
dtype

①把几个id的格式转化成object格式

In [21]:
data.order_id = data.order_id.astype('object')
data.product_id = data.product_id.astype('object')
data.category_id = data.category_id.astype('object')
data.user_id = data.user_id.astype('object')

②把weekday转化成int格式

In [22]:
data['weekday'] = data.loc[:,'weekday'].astype('int')

In [28]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 536311 entries, 0 to 536310
Data columns (total 15 columns):
 #   Column         Non-Null Count   Dtype         
---  ------         --------------   -----         
 0   order_id       536311 non-null  object        
 1   product_id     536311 non-null  object        
 2   category_id    536311 non-null  object        
 3   category_code  536311 non-null  object        
 4   brand          536311 non-null  object        
 5   price          536311 non-null  float64       
 6   user_id        536311 non-null  object        
 7   age            536311 non-null  float64       
 8   sex            536311 non-null  object        
 9   local          536311 non-null  object        
 10  date           536311 non-null  datetime64[ns]
 11  month          536311 non-null  int64         
 12  weekday        536311 non-null  int32         
 13  buy_cnt        536311 non-null  int64         
 14  amount         536311 non-null  float64       
dtype

③使用统计方法查看数值型变量（价格和年龄）是否异常

In [23]:
data.describe(percentiles=[0.01,0.25,0.75,0.99]).T

,count,mean,std,min,1%,25%,50%,75%,99%,max
price,536311.00,214.54,305.98,0.00,1.13,24.51,99.51,289.33,1387.01,11574.05
age,536311.00,33.18,10.12,16.00,16.00,24.00,33.00,42.00,50.00,50.00
month,536311.00,7.72,2.56,1.00,1.00,6.00,8.00,10.00,11.00,11.00
weekday,536311.00,3.03,2.04,0.00,0.00,1.00,3.00,5.00,6.00,6.00
buy_cnt,536311.00,1.00,0.04,1.00,1.00,1.00,1.00,1.00,1.00,4.00
amount,536311.00,214.73,306.48,0.00,1.13,24.98,99.51,289.33,1387.01,11574.05


以上7个字段均无异常情况：
price和amount最小值为0，可能是免费商品或者赠品，不属于异常情况

④检查其他字段是否有异常

In [24]:
data.describe(include='all').T

,count,unique,top,freq,first,last,mean,std,min,25%,50%,75%,max
order_id,536311.00,389511.00,2388440981134689792.00,47.00,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN
product_id,536311.00,19078.00,1515966223517847040.00,2755.00,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN
category_id,536311.00,786.00,2268105428166508800.00,101081.00,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN
category_code,536311,124,M,116093,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN
brand,536311,868,samsung,96123,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN
price,536311.00,NaN,NaN,NaN,NaT,NaT,214.54,305.98,0.00,24.51,99.51,289.33,11574.05
user_id,536311.00,92769.00,1515915625512422912.00,1026.00,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN
age,536311.00,NaN,NaN,NaN,NaT,NaT,33.18,10.12,16.00,24.00,33.00,42.00,50.00
sex,536311,2,男,270454,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN
local,536311,11,广东,117097,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN


发现date日期有1970-01-01的情况，应该属于异常情况，对其删除

In [25]:
data = data[data.date>'1970-01-01']

In [26]:
data.date.min()

Timestamp('2020-01-05 00:00:00')

### 3 数据清洗完成 

In [27]:
data.head()

,order_id,product_id,category_id,category_code,brand,price,user_id,age,sex,local,date,month,weekday,buy_cnt,amount
0,2318945879811162983,2309018204833317816,2268105479144079872,M,compliment,0.56,1515915625465863936,28.00,女,浙江,2020-05-28,5,4,4,2.24
1,2295740594749702229,1515966223509104892,2268105428166508800,electronics.smartphone,apple,1387.01,1515915625448766464,21.00,男,北京,2020-04-26,4,0,4,5548.04
2,2388440981134674698,1515966223509106757,2360741867017995776,appliances.environment.air_conditioner,samsung,366.41,1515915625514599680,50.00,男,广东,2020-11-16,11,1,4,1465.64
3,2375043331555066740,2273948308370096764,2268105409048871168,computers.network.router,altel,57.85,1515915625504379136,19.00,女,上海,2020-08-13,8,4,4,231.40
4,2334999887038383089,1515966223509090031,2268105402673529600,M,vitek,18.50,1515915625447765248,18.00,男,广东,2020-06-19,6,5,3,55.50


In [28]:
data.shape
#共计53w行，15列数据

(535065, 15)

In [29]:
data.reset_index(drop=True,inplace=True)

In [30]:
#保存
data.to_csv('data_clean.csv',index = False)

In [8]:
df1 = pd.read_csv('data_clean.csv')

In [9]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 535065 entries, 0 to 535064
Data columns (total 15 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   order_id       535065 non-null  int64  
 1   product_id     535065 non-null  int64  
 2   category_id    535065 non-null  int64  
 3   category_code  535065 non-null  object 
 4   brand          535065 non-null  object 
 5   price          535065 non-null  float64
 6   user_id        535065 non-null  int64  
 7   age            535065 non-null  float64
 8   sex            535065 non-null  object 
 9   local          535065 non-null  object 
 10  date           535065 non-null  object 
 11  month          535065 non-null  int64  
 12  weekday        535065 non-null  int64  
 13  buy_cnt        535065 non-null  int64  
 14  amount         535065 non-null  float64
dtypes: float64(3), int64(7), object(5)
memory usage: 61.2+ MB


In [10]:
df1.shape

(535065, 15)